# Text Classification using Simple Transformers (NLP)

- This code was used to classify text messages as spam or not. The challenge was part of a class assignment at Queen's University. The beauty about Simple Transformers is that it takes very few lines of code to create your model. It's very similar to the PyCaret package.

### 1. Install Packages

In [7]:
#Create a new virtual environment and install packages
conda create -n st python pandas tqdm -y

SyntaxError: invalid syntax (<ipython-input-7-44992e147628>, line 2)

In [3]:
#Create a new virtual environment and install packages
conda activate st

SyntaxError: invalid syntax (<ipython-input-3-77e1412671a6>, line 2)

In [4]:
#Install Simple Transformers
pip install simpletransformers

SyntaxError: invalid syntax (<ipython-input-4-d7696b000ca1>, line 2)

In [8]:
#Boring install stuff
conda install pytorch torchvision -c pytorch

SyntaxError: invalid syntax (<ipython-input-8-fec855080e21>, line 2)

In [9]:
#Import the ST Classification Model and Data Wrangling Packages
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [10]:
#Log Results
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### 2. Import and Transform Data

In [11]:
#Import Train and Test Datasets
train_df = pd.read_csv("/Users/williamchan/Desktop/spamraw_train.csv")
test_df = pd.read_csv("/Users/williamchan/Desktop/spamraw_test.csv")

In [12]:
#Drop ID columns
train_df.drop(columns = 'id')
test_df.drop(columns = 'id')

,sms_text
0,HOT LIVE FANTASIES call now 08707509020 Just 2...
1,I not at home now lei...
2,So how's scotland. Hope you are not over showi...
3,Yo you around? A friend of mine's lookin to pi...
4,I'm aight. Wat's happening on your side.
...,...
554,You are a great role model. You are giving so ...
555,"Awesome, I remember the last time we got someb..."
556,"If you don't, your prize will go to another cu..."
557,"SMS. ac JSco: Energy is high, but u may not kn..."


In [13]:
#Set text field as string
train_df['sms_text'] = train_df['sms_text'].astype('str')

In [14]:
#Rename data field for model
## Reference: https://simpletransformers.ai/docs/classification-data-formats/
train_df = train_df.rename(columns={"sms_text": "text", "spam": "labels"})

In [15]:
#Rename data field for model
## Reference: https://simpletransformers.ai/docs/classification-data-formats/
test_df = test_df.rename(columns={"sms_text": "text"})

In [16]:
#Check dataframe type
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5000 non-null   int64 
 1   text    5000 non-null   object
 2   labels  5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [17]:
#Check dataframe
train_df.head()

,id,text,labels
0,1,Hope you are having a good week. Just checking in,0
1,2,K..give back my thanks.,0
2,3,Am also doing in cbe only. But have to pay.,0
3,4,"complimentary 4 STAR Ibiza Holiday or £10,000 ...",1
4,5,okmail: Dear Dave this is your final notice to...,1


### 3. Create Classification Model

- I chose to use the roBERTa classification model. SimpleTransformers supports other model types like ALBERT, BERT, or BERTweet (for English Tweets). Full list of [models](https://simpletransformers.ai/docs/classification-specifics/).

In [19]:
#Set and create the classification model we want to use
model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [20]:
#Train the classification model using training data
model.train_model(train_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.



/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning:

Please also save or load the state of the optimzer when saving or loading the scheduler.



INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


### 4. Evaluate the Model

- Our trained model was able to produce an accuracy of 99.3%. It accurately identified 656 spam messages, accurately identified 4311 non-spam messages, falsely identified 16 messages as spam, and 17 messages as non-spam.

In [22]:
#Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(train_df, acc=sklearn.metrics.accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.9716522200041187, 'tp': 656, 'tn': 4311, 'fp': 16, 'fn': 17, 'acc': 0.9934, 'eval_loss': 0.033143126635812224}


### 5. Make Predictions on Test Data and Export to Kaggle

In [23]:
#Run the model on test data
predictions, raw_outputs = model.predict(test_df['text'])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [25]:
#Export Predictions for Kaggle
my_submission = pd.DataFrame({'id': test_df.id, 'predicted': predictions})
my_submission.head()
my_submission.to_csv('my_submission.csv', index=False)